# Utilitários

## Copia arquivo para servidor remoto

### Instala sshpass para passar a senha para o scp

In [ ]:
!apt install sshpass

### Executa scp para copiar arquivo

In [ ]:
user = ""
senha = ""
host = "rafaelescalfoni.net"
file = "prefeito_rio_2020_pt.csv"
# Executa comando no bash
!sshpass -p $senha scp -o 'StrictHostKeyChecking no' $file $user@$host:

# Bibliotecas

In [ ]:
import pandas as pd
import json
import nltk
import re
# Imports individuais
from pandas.core.frame import DataFrame
from nltk import tokenize
from nltk.corpus import stopwords
from string import punctuation
# NLTK corpora
nltk.download('punkt')
nltk.download('stopwords')

# Abre o DataSet diretamente da internet

Desabilita verificação SSL em caso de certificado auto-assinado:

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

Lê o Dataset Original:

In [ ]:
url = "https://rafaelescalfoni.net/prefeito_rio_2020.csv"
ds = pd.read_csv(url)
ds.info()

Lê o Dataset Limpo:

In [ ]:
url = "https://rafaelescalfoni.net/prefeito_rio_2020_clean.csv"
# Parâmetro "na_filter" usado para lidar com campos nulos como string
ds = pd.read_csv(url, na_filter=False)
ds.info()

Lê o Dataset com Texto Processado:

In [ ]:
url = "https://rafaelescalfoni.net/prefeito_rio_2020_pt.csv"
# Parâmetro "na_filter" usado para lidar com campos nulos como string
ds_por = pd.read_csv(url, index_col=0, na_filter=False)
ds_por.info()

# Limpeza de Dados

## Eliminação de colunas

### Função para eliminar colunas com taxa de nulos acima do limite

In [ ]:
def drop_column_null_above_limit(df: DataFrame, ignored_columns=[], threshold=0.8):
  columns = df.columns.drop(ignored_columns)
  # Pega colunas acima do limite de null, exceto as ignoradas
  columns = [column for column in columns if df[column].isna().sum()/df.shape[0] > threshold]

  return df.drop(columns, axis=1)

### Elimina colunas com alta taxa de nulos

In [ ]:
# Resguarda coluna "extended_tweet" de eliminação por conta da análise de sentimento
ds = drop_column_null_above_limit(ds, ['extended_tweet'])
ds.info()

### Função para eliminar colunas com um único valor

In [ ]:
def drop_column_unique_value(df: DataFrame):
  # Pega colunas com um único valor
  columns = [column for column in df.columns if df[column].nunique() == 1]

  return df.drop(columns, axis=1)

### Elimina colunas com um único valor

In [ ]:
ds = drop_column_unique_value(ds)
ds.info()

### Elimina as colunas que não têm utilidade para nossa análise

In [ ]:
ignored_columns = ['_id', 'id_str']
ds.drop(columns=ignored_columns, inplace=True)
ds.info()

## Tratamento das colunas JSON

### Função para converter listas para string JSON no DataFrame

In [ ]:
def converte_lista_json(df: DataFrame):
  for column in df.columns:
    df[column] = df[column].apply(lambda x: json.dumps(x) if type(x) == list else x)

### Lista colunas com objetos JSON

In [ ]:
# Função que verifica se determinada string é um objeto json (parâmetro opcional = somente dicionário)
def is_json(myjson, ignore_list=True):
  try:
    json_object = json.loads(myjson)
  except:
    return False
  # Verifica se o objeto json é uma lista
  if (ignore_list and type(json_object) == list):
    return False
  else:
    return True
# Analisa sempre o primeiro índice não nulo da referida coluna, ou 0 quando a coluna é vazia
[column for column in ds.columns if is_json(ds[column][ds[column].first_valid_index() or 0])]

### Expandir colunas JSON

#### Coluna user

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
expanded_user = pd.json_normalize(ds.user.apply(json.loads))
expanded_user.info()

Elimina colunas com alta taxa de nulos:

In [ ]:
expanded_user = drop_column_null_above_limit(expanded_user)
expanded_user.info()

Elimina colunas com um único valor:

In [ ]:
expanded_user = drop_column_unique_value(expanded_user)
expanded_user.info()

Elimina as colunas que não têm utilidade para nossa análise:

In [ ]:
ignored_columns = ['id', 'url', 'translator_type', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https',
                   'profile_background_tile', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color',
                   'profile_use_background_image', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'default_profile', 'id.$numberLong']
expanded_user.drop(columns=ignored_columns, inplace=True)
expanded_user.info()

#### Coluna extended_tweet

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
# Transforma dados null em objetos json vazio {} antes de carregar
expanded_ext_tweet = pd.json_normalize(ds.extended_tweet.fillna('{}').apply(json.loads))
expanded_ext_tweet.info()

Elimina colunas com alta taxa de nulos:

In [ ]:
# Resguarda colunas de eliminação por conta de análises
expanded_ext_tweet = drop_column_null_above_limit(expanded_ext_tweet, ['full_text', 'entities.hashtags', 'entities.urls', 'entities.user_mentions'])
expanded_ext_tweet.info()

Converte colunas contendo listas em representações string de objetos JSON:

In [ ]:
converte_lista_json(expanded_ext_tweet)

#### Coluna entities

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
expanded_entities = pd.json_normalize(ds.entities.apply(json.loads))
expanded_entities.info()

Elimina colunas com alta taxa de nulos:

In [ ]:
expanded_entities = drop_column_null_above_limit(expanded_entities)
expanded_entities.info()

Elimina as colunas que não têm utilidade para nossa análise:

In [ ]:
ignored_columns = ['symbols']
expanded_entities.drop(columns=ignored_columns, inplace=True)
expanded_entities.info()

Converte colunas contendo listas em representações string de objetos JSON:

In [ ]:
converte_lista_json(expanded_entities)

## Faz o Join desses datasets com o principal

In [ ]:
# Adiciona prefixo "user_" às colunas do DS "expanded_user" antes do join
ds = ds.join(expanded_user.add_prefix('user_'))
# Adiciona prefixo "tweet_" às colunas do DS "expanded_ext_tweet" antes do join
ds = ds.join(expanded_ext_tweet.add_prefix('tweet_'))
# Adiciona prefixo "entities_" às colunas do DS "expanded_entities" antes do join
ds = ds.join(expanded_entities.add_prefix('entities_'))
# Elimina as colunas JSON
ds.drop(columns=['user', 'extended_tweet', 'entities'], inplace=True)
ds.info()

## Troca as colunas relativas ao tweet estendido quando o tamanho ultrapassa 140 caracteres

In [ ]:
# Colunas referentes ao tweet original (140) e estendido (280)
columns_140 = ['text', 'entities_hashtags', 'entities_urls', 'entities_user_mentions']
columns_280 = ['tweet_full_text', 'tweet_entities.hashtags', 'tweet_entities.urls', 'tweet_entities.user_mentions']
# Troca os valores das colunas originais pelos das colunas estendidas
ds.loc[ds.truncated, columns_140] = ds.loc[ds.truncated, columns_280].values

## Elimina as colunas relativas ao tweet estendido

In [ ]:
ds.drop(columns=columns_280, inplace=True)
ds.info()

## Preenchimento de nulos

In [ ]:
# Preenche campos texto com a string vazia
ds['source'] = ds['source'].fillna('')
ds['user_location'] = ds['user_location'].fillna('')
ds['user_description'] = ds['user_description'].fillna('')

## Salva Dataset limpo

In [ ]:
# Parâmetro "escapechar" usado para lidar com quebra de linhas nos campos texto
ds.to_csv('prefeito_rio_2020_clean.csv', index=False, escapechar='\r')

# Processamento de Texto

## Função de Tokenização

In [ ]:
def clean_and_tokenize(text: str, entities_urls: list, language='portuguese', check_url=True):
  texto = text
  # Executa limpeza de urls no texto
  if check_url:
    for urls in entities_urls:
      texto = texto.replace(urls.get('url'), '')
  # Tokenização por palavras e limpeza de stopwords, pontuação e símbolos
  palavras = tokenize.word_tokenize(texto.lower(), language)
  stop_words = set(stopwords.words(language) + list(punctuation))
  palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stop_words and re.search("\w+", palavra)]

  return palavras_sem_stopwords

## Carrega strings JSON como objetos

In [ ]:
ds.entities_hashtags = ds.entities_hashtags.apply(json.loads)
ds.entities_urls = ds.entities_urls.apply(json.loads)
ds.entities_user_mentions = ds.entities_user_mentions.apply(json.loads)

## Cria dataset tokenizado em português

In [ ]:
ds_por = ds[ds.lang == 'pt']
text_tokens = ds_por.apply(lambda row: clean_and_tokenize(row['text'], row['entities_urls']), axis=1)
text_tokens.name = 'text_tokens'
ds_por = ds_por.join(text_tokens)
ds_por.head()

### Tokeniza Descrição do Usuário

In [ ]:
text_tokens = ds_por.user_description.apply(lambda row: clean_and_tokenize(row, [], check_url=False))
text_tokens.name = 'user_description_tokens'
ds_por = ds_por.join(text_tokens)
ds_por.head()

### Salva Dataset em Português

In [ ]:
# Parâmetro "escapechar" usado para lidar com quebra de linhas nos campos texto
ds_por.to_csv('prefeito_rio_2020_pt.csv', escapechar='\r')

# TODO Análise Exploratória de Dados

In [ ]:
ds_ing = ds[ds.lang == 'en']
ds_und = ds[ds.lang == 'und']
ds_ing.head()
ds_und.head()

In [ ]:
ds['lang'].value_counts()